# Exercise 5: Biostatistical analysis

Our previous survival analyses highlighted that the drugB could have a limited effect on several subgroups of the population (ex1). Therefore, clinicians asked us to dig further the analysis on this medication and try to quantify its effect on death event, along with other clinical and biological features. To that end, we will implement Multiple Linear Regression models, which allow to "associate" an event (*e.g.*, death) to a pre-defined set of quantitative and qualitative features.

A restricted view of the database has been constructed to remove information about the patients prescribed with drugA, so that we can focus on drugB vs control population.
Let's assume for this exercise that this view has also been cleaned up by a previous pre-processing work (cf ex 1,2,3). We will focus on merging the different tables available into one that we will use to perform our statistical analyses.

The following libraries are required to go through this notebook:

In [ ]:
import pandas as pd
import numpy as np

# For descriptive statistics
from scipy.stats import ttest_ind, chi2_contingency

# For Logistic Models implementation
from statsmodels.formula.api import logit, glm

# Table of content

1. [Preprocessing](#preprocessing)  
    1.1 [Age](#age)  
    1.2 [Gender](#gender)  
    1.3 [Death](#death)  
    1.4 [Comorbidities](#comorb)  
    1.5 [Biological values](#bio)  
    1.6 [Medication data](#med)
2. [Descriptive statistics](#stat)  
3. [Statistical analysis](#analysis)  
    3.1 [Univariate analaysis](#uni_analysis)  
    3.2 [Generalized Linear Model](#multi_analysis)  
4. [Takeaways](#takeaways)

<a id="preprocessing"></a>
# 1. Preprocessing

The main objective of this step is to prepare a ready-to-use table for the subsequent statistical analyses. We want to summarize the demographic, clinical and biological information of each patient in the cohort.

Hence, the output DataFrame will include one row by patient, having the following columns:
- age at the time of the visit (*int*)
- male (*bool*)
- death at the time of the study (*bool*). We assume that the study is realized long enough after the visits so that we can overlook censoring issues.
- comorbidities : history of cancer and diabetes (*bool*)
- biological profile at the time of admission (*float*). The following items were measured for each patient : bmi, urea, hemoglobin, crp.
- drubB taken (*bool*)

**NB: For the sake of simplicity, we assume that all the available tables have been pre-processed (deduplication, NLP enhancing , data cleaning, ...) and can be used as is. Moreover, we assume that each patient has one and only visit in the database.**

Open the following files using the `pandas.read_pickle()` function : 
  - *data/df_person.pkl* as `df_person`
  - *data/df_visit.pkl* as `df_visit`
  - *data/df_condition.pkl* as `df_cond`
  - *data/df_med.pkl* as `df_med`
  - *data/df_bio.pkl* as `df_bio.pkl`

In [ ]:
# Patients
df_person = pd.read_pickle('data/df_person.pkl')
# Visits
df_visit = pd.read_pickle('data/df_visit.pkl')
# Diagnosis (condition)
df_condition = pd.read_pickle('data/df_condition.pkl')
# Medication
df_med = pd.read_pickle('data/df_med.pkl')
# Biology
df_bio = pd.read_pickle('data/df_bio.pkl')

Create the base output dataset (format : `pandas.DataFrame` ; name : `data`)  from the `df_person` table, gathering patient id, gender, birth and death dates.

In [ ]:
data = df_person[["person_id", "gender_source_value", "birth_datetime", "death_datetime"]]

In [ ]:
data.head()

Check the size of the dataframe using `dataframe.shape()`

In [ ]:
data.shape

<a id="age"></a>
## 1.1 Age

Compute the patient age at the time of their hospitalization.

TIPS : 
- Inner join the `data` dataset and the `df_visit` dataset on `person_id` using the `pandas.merge()` function (see [doc](https://pandas.pydata.org/docs/reference/api/pandas.merge.html))
- Create an `age` column compute the age for each patient.

In [ ]:
data = pd.merge(data, df_visit[["person_id", "visit_occurrence_id", "visit_start_datetime"]], on="person_id", how="inner")

In [ ]:
data.head()

In [ ]:
data['age'] = round((data.visit_start_datetime - data.birth_datetime).dt.days/365)

<a id="gender"></a>
## 1.2 Gender

Create a "male" boolean column to identify if the patient is a male (1) or a female (0).

In [ ]:
data["male"] = data.gender_source_value == 'm'

<a id="death"></a>
## 1.3 Death

Create a "dead" boolean column to identify if the patient has died or not.

TIP : The patient is dead if a date of death is mention. Use the function `pandas.notna()`

In [ ]:
data.head()

In [ ]:
data["dead"] = data.death_datetime.notna()

<a id="comorb"></a>
## 1.4 Comorbidities : history of cancer and diabetes

Meetings with experts lead us to identify the ICD-10 codes corresponding to history of cancer and diabetes :
- cancer : ["Z851", "Z852", "Z853"]
- diabetes : ["E10", "E11", "E12"]

Create a `comorbidity` column by tagging each code in the `df_cond` Dataframe corresponding to diabetes or cancer antecedent.

TIP: you can build a `code_to_comorb` dictionary mapping each ICD-10 code to its corresponding comorbidity, and use the pandas `df.col_name.map(dict)` built-function to map the values of the `condition_source_value` column.

In [ ]:
comorb_to_code = {
    "diabetes": ["E10", "E11", "E12"],
    "history_cancer": ["Z851", "Z852", "Z853"]
}

In [ ]:
code_to_comorb = {}
for key,codes_list in comorb_to_code.items():
    print(key, codes_list)
    for code in codes_list:
        code_to_comorb[code] = key

In [ ]:
code_to_comorb

In [ ]:
df_condition["comorbidity"] = df_condition.condition_source_value.map(code_to_comorb)

In [ ]:
df_condition.sample(10)

Create a `df_comorbidities` DataFrame by selecting the condition codes corresponding to the desired comorbidities.

TIP : 
You can use the `df.col_name.notna()` built-in function : 
```python 
df[df.col_name.notna()]
```

In [ ]:
df_comorbidities = df_condition[df_condition.comorbidity.notna()]

How many patient have cancer antecedent ? How many have had diabetes ?

TIP : Use the `df.col_name.value_counts()` function

In [ ]:
df_comorbidities.comorbidity.value_counts()

In [ ]:
df_comorbidities.head()

The `df_comorbidities` DataFrame is composed of one row per condition source value (either cancer or diabete).
<br> A patient can have multiple rows depending on how many conditions they have.

Now, we will create a dataframe with one row per patient and per visit : 
<br>NB : each patient only has one and only visit

| visit_occurrence_id | person_id  | history_cancer | diabetes |
| :------------------ | :----------|:------------| :---------- |
| aaa | xxx | True |  True |
| bbb | yyy | False |  True |
| ccc | zzz | False |  False |

Pivot the `df_comorbidities` DataFrame to have the presence of both antecedents for each patient. Don't forget to replace the NaN values by False and to reset the index !

TIP 
You can use the [pivot](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html) built-in function :
```python
df.pivot(index=["index_col"], columns=["col"], values=["value"]])
```
You might need to create "artificially" the *value* column as a True boolean, and drop the *condition_occurrence_id* column.

In [ ]:
df_comorbidities["value"] = True

In [ ]:
df_comorbidities = df_comorbidities.drop(columns=["condition_occurrence_id"]).pivot(index=["visit_occurrence_id", "person_id"], columns="comorbidity", values="value").reset_index().fillna(False)

In [ ]:
df_comorbidities.head()

Merge the `data` DataFrame with the obtained `df_comorbidities` to summarize comorbidities information for each patient.

TIP:  
- Think twice about the type of merging you want to perform
- Do not forget to fill NA values for the comorbidity columns with False !

In [ ]:
df_comorbidities.person_id.nunique()

In [ ]:
data.person_id.nunique()

In [ ]:
data = pd.merge(data, df_comorbidities[["person_id", "history_cancer", "diabetes"]], on="person_id", how="left").fillna({"diabetes":False, "history_cancer":False})

In [ ]:
data.head()

<a id="bio"></a>
## 1.5 Biological profile

Check the different biological values available.

In [ ]:
df_bio.concept_source_value.value_counts()

In [ ]:
df_bio.head()

Create a `df_bio_processed` DataFrame with all biological values for each visit : 

| visit_occurrence_id | bio_A  | bio_B | bio_C |
| :------------------ | :----------|:------------| :---------- |
| aaa | 189 | 13 |  0.87 |
| bbb | 160 | 11 |  0.59 |
| ccc | 209 | 11 |  0.69 |


TIP : Pivot the `df_bio` DataFrame and retrieve the biological values measured at the admission of each hospitalization.

In [ ]:
bio_processed = df_bio[["visit_occurrence_id", "concept_source_value", "transformed_value"]].pivot(index=["visit_occurrence_id"], columns="concept_source_value", values="transformed_value").reset_index()

In [ ]:
bio_processed.head()

Merge `data` and `bio_processed` on `visit_occurrence_id`

TIP: Think twice about the type of merging you want to perform

In [ ]:
data.head()

In [ ]:
data = pd.merge(data, bio_processed, on="visit_occurrence_id", how="left")

In [ ]:
data.head()

<a id="med"></a>
## 1.6 Medication taken

In [ ]:
df_med.head()

Create a "drugB" boolean column in `data` indicating whether patients have taken the drugB or not. 

TIP :
You can pivot the `df_med` DataFrame into a `med_processed` dataframe as done for the comorbidities and merge it with `data`.

In [ ]:
df_med["value"]=True

In [ ]:
med_processed = df_med[["visit_occurrence_id", "drug_source_value", "value"]].pivot(index="visit_occurrence_id", columns="drug_source_value", values="value").reset_index()

In [ ]:
med_processed.head()

In [ ]:
med_processed.drugB.value_counts()

In [ ]:
data = pd.merge(data, med_processed, on="visit_occurrence_id", how='left').fillna({"drugB":False})

In [ ]:
data.head()

In [ ]:
data.drugB.value_counts()

### Final step

Select the columns you want to keep for the statistical analyses and check that you don't have missing values.

TIP : you can use the following built-in function :
```python
df.isna().sum()
```
Beware of the axis along which you sum !

In [ ]:
data = data[["person_id", "age", "male", "dead", "history_cancer", "diabetes", "bmi", "crp", "urea", "hb", "drugB"]]

In [ ]:
data.isna().sum(axis=0)

<a id="stat"></a>
# 2. Descriptive statistics

Let's compute base statistics for our features in order to catch a glimpse of our population characteristics by separating dead vs alive patients. We will compute :
- For quantitative features : mean (std) and student t-test
- For qualitative features : % of prevalence and Chi-sq test

*Assuming that we are respecting the initial conditions of each test (iid gaussian distribution for quanti feats).*

We are going to use the `df.groupby().agg(agg_dict)` function. Check out the doc [here](https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.core.groupby.DataFrameGroupBy.agg.html)  
We will therefore need to provide for each column of `data` the desired aggregation function.

Create a "prevalence" function that returns the percentage of True values from a list of booleans.

In [ ]:
def prevalence(values):
    return values.sum()*100/len(values)

Create the "agg_dict" dictionary storing for each column name (keys) the list of transforming functions (values). You can use the `np.mean` and `np.std` registered functions.

TIP : Example
```python
agg_dict = {
    "person_id": ["count"],
    "age": [np.mean, np.std],
    ...
}
```

In [ ]:
agg_dict = {
    "person_id": ["count"],
    "age": [np.mean, np.std],
    "male": [prevalence],
    "history_cancer" : [prevalence],
    "diabetes" : [prevalence],
    "drugB": [prevalence],
    "bmi" : [np.mean, np.std],
    "crp": [np.mean, np.std],
    "hb": [np.mean, np.std],
    "urea": [np.mean, np.std],
}

Create a summary DataFrame from `data` using the predefined `agg_dict` to compare base statistics between the living and the dead groups.

In [ ]:
data.head(2)

In [ ]:
data_summary = round(data.groupby("dead").agg(agg_dict).T,2)

In [ ]:
data_summary

Compute statistical tests to assess the significancy of the resulting difference between the dead and the living groups.  
For numerical values (age and bio), you can compute the p-values associated to Student t-tests (function `ttest_ind`, doc [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)), and for categorical values (gender, comorbidities and drug) chi-sq tests (function `chi2_contigency`, doc [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html)).

TIP : 
You may need to separate `data` into `data_alive` and `data_dead` in order to facilitate the statistic computation.

In [ ]:
numerical = ["age", "bmi", "crp", "hb", "urea"]
test_categorical = ["male", "history_cancer", "diabetes", "drugB"]

In [ ]:
data_alive = data.query("dead==False")
data_dead = data.query("dead==True")

In [ ]:
data_summary.index

In [ ]:
chi2_contingency(pd.crosstab(data["dead"], data["drugB"]))

In [ ]:
for feat, sub_value in data_summary.index:
    if sub_value == 'std':
        continue
    if feat in numerical:
        data_summary.loc[(feat,sub_value),"t-test"] = round(ttest_ind(data_alive[feat], data_dead[feat])[1],4)

    if feat in test_categorical:
        data_summary.loc[(feat,sub_value),"chi-sq"] = round(chi2_contingency(pd.crosstab(data["dead"], data[feat]))[1],4)


In [ ]:
data_summary

What can you conclude ?

Dead patients appear to be older, having diabetes more frequently and presenting a worse biological profile (especially regarding Urea) at admission.
**However**, the statistical significance must be confronted to the sample size. Indeed, with >20000 patients, the statistical tests are de facto significant, and those p-values are "artificially" low.

<a id="analysis"></a>
# 3. Statistical analysis

Now that we have a better overview of our cohort and its characteristics, we can perform the desired statistical analysis by fitting logistic regression models.

Convert binary features into integer, as boolean are not supported by the statsmodels functions.

In [ ]:
categorical = ["dead","male", "atcd_cancer", "diabetes", "drugB"]
data[categorical] = data[categorical].astype(int)

<a id="uni_analysis"></a>
## 3.2 Univariate logistic regression : OR associated with taking drugB

In [ ]:
data.head()

Build a logistical regression model to evaluate the univariate effect of taking drug B on death, and fit it on `data`. Documentation about the `logit` function can be found [here](https://www.statsmodels.org/devel/generated/statsmodels.formula.api.logit.html).

In [ ]:
model = logit("dead~drugB",data).fit(disp=False)
print(model.summary())

Extract the coefficient associated with drugB using `model.params[param_name]`

In [ ]:
model.params["drugB"]

Compute the Odds Ratio associated with the drugB feature.

TIP :
OR = exp(feature_coef)

In [ ]:
or_uni = np.exp(model.params["drugB"])

In [ ]:
or_uni

What can you conclude of the odds of death when taking drugB ?

In [ ]:
print(f"According to our univariate analysis, the odds of dying appears to be multiplied by {round(or_uni,2)}, or reduced by {1/or_uni} when taking drug B.")

Perform the same univariate analysis on the other features relating to death event.

In [ ]:
for feat in ["age", "male", "atcd_cancer", "diabetes", "urea", "crp", "bmi", "hb"]:
    model = logit(f"dead ~  {feat}", data).fit(disp=False)
    print(f"Univariate OR associated with {feat}: {round(np.exp(model.params[feat]),2)}")


What parameters seem to have the greatest influence on the death event ?

The most influent parameters on death are the following :
#TODO

<a id="multi_analysis"></a>
## 3.3 Multivariate analysis : Generalized Linear Models

Build a Generalized Linear Model to explain the death event with all the available features. Documentation about the `glm` function can be found [here](https://www.statsmodels.org/devel/generated/statsmodels.formula.api.glm.html).

In [ ]:
model = glm("dead ~ drugB + age + male + history_cancer + diabetes + urea + crp + bmi + hb", data).fit()
print(model.summary())

Compute the stratified OR associated with taking drugB :

In [ ]:
or_multi = np.exp(model.params["drugB"])

Compute the OR associated with each feature after adjustement.

In [ ]:
for feat, param in model.params.to_dict().items():
    print(feat, round(np.exp(model.params[feat]),2))

What can you conclude ?

In [ ]:
print(f"After a stratification on other features, the odds of death appear to be reduced by  {#TODO} when taking drug B.")

The most impactful features on the adjusted model is:
- #TODO

**CONCLUSION**
The effect of drug B adjusted on all available features is reduced, illustrating the fact that this medication was more frequently prescribed to younger people in better global health (less comorbidities and better biological features). 
Moreover, the effect of the other features (urea, comorbidities, etc..) vanishes when adjusting on drugB.  
This is a typical clinical bias that can be found frequently in Real-World Data and may lead to over-interpretation of drug effects... do not conclude to quickly !

# 4. Takeaways

Large scale Real-World Data may be subject to major clinical biases, such as prescription bias. They can affect the overall population, or be subject to differing guidelines and practices among the diversity of hospitals, clinicians and patients. 
It is of paramount importance to work closely with clinician to be aware of such biases and avoid over-interpretation, as long as performing stratified analyses to have a broader view of the effects.